## W&B Loading

In [ ]:
from modelzoo.utils.io_model import load_wandb_ckpt

# Select your favorite run id from the wandb dashboard
# https://wandb.ai/gladia/modelzoo/table?workspace=user-lucmos
ENTITY = "gladia"
PROJECT = "modelzoo"
RUN_ID = "04xt6ch5"

# Load the remote model
model, ckpt = load_wandb_ckpt(ENTITY, PROJECT, RUN_ID)
model

## Local loading

In [ ]:
from nn_core.common import PROJECT_ROOT
from modelzoo.utils.io_model import load_local_ckpt

filepath = PROJECT_ROOT / "models" / "checkpoints" / "7v62350u.ckpt.zip"
model, ckpt = load_local_ckpt(filepath)
model

## Extract configuration from ckpt

In [ ]:
from omegaconf import OmegaConf

cfg = OmegaConf.create(ckpt["cfg"])

In [ ]:
from torch.utils.data import DataLoader
import hydra
from functools import partial
from modelzoo.data.vision.datamodule import collate_fn
from omegaconf import OmegaConf

# Instantiate validation and traing loaders, with correct transforms
datamodule = hydra.utils.instantiate(OmegaConf.to_container(cfg.nn.data), _recursive_=False)
datamodule.setup(stage="fit")
train_dataset = datamodule.train_dataset
val_dataset = datamodule.val_datasets[0]

train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    pin_memory=True,
    shuffle=False,
    num_workers=4,
    collate_fn=partial(collate_fn, split="train", metadata=datamodule.metadata, transform=datamodule.transform_batch),
)
val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    pin_memory=True,
    shuffle=False,
    num_workers=4,
    collate_fn=partial(collate_fn, split="val", metadata=datamodule.metadata, transform=datamodule.transform_batch),
)

train_dataset, val_dataset

In [ ]:
# Test inference on a batch

batch = next(iter(train_loader))
model(batch["x"])